# 5장

### RNN 코드 구현 

In [4]:
class RNN:
    def __init__(self, Wx, Wh, b):
        # 가중치 2개, 편향 1개
        self.params = [Wx, Wh, b]
        # 각 매개변수에 대응하는 형태로 기울기 초기화 후 grads에 저장
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        # 역전파 계산시 중간 데이터를 담은 cache를 None으로 초기화
        self.cache = None
    
    def forward(self, x, h_prev):
        # 아래로부터 입력 x와 왼쪽으로부터의 입력 h_prev
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next*(1-h_next**2)
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.T)
        dWx = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

### Time RNN

In [ ]:
#RNN클래스를 이용해 T개 단계의 처리를 한꺼번에 수행하는 계층을 
#TimeRNN이란 이름의 클래스로 완성한다.
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=false):
        #초기화 메서드는 가중치, 편향, stateful이라는 boolean값을 인수로 받음
        #stateful=True : Time RNN계층이 은닉 상태를 유지한다.->아무리 긴 시계열 데이터라도 Time RNN계층의 순전파를 끊지 않고 전파한다.
        #stateful=False: Time RNN 계층은 은닉 상태를 '영행렬'로 초기화한다.상태가 없다.
        self.params = [Wx, Wh, b]
        self.grads = [np.zeors_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        #layers : 다수의 RNN계층을 리스트로 저장하는 용도

        self.h, self.dh = None, None
        #h: forward() 메서드를 불렀을 때 마지막 RNN 계층의 은닉 상태를 저장
        #dh: backward()를 불렀을 때 하나 앞 블록의 은닉 상태의 기울기를 저장한다.
        self.stateful = stateful

    def set_state(self, h):
        #Time RNN계층의 은닉 상태를 설정하는 메서드
        self.h = h

    def reset_state(self):
        #은닉 상태를 초기화하는 메서드
        self.h = None

#순전파 구현
def forward(self, xs):
    #아래로부터 입력 xs(T개 분량의 시계열 데이터를 하나로 모은 것)를 받는다.
    Wx, Wh, b = self.params
    N, T, D = xs.shape #N: 미니배치 크기 D: 입력 벡터의 차원 수
    D, H = Wx.shape

    self.layers = []
    hs = np.empty((N, T, H), dtype='f')
    #출력값을 담을 그릇 hs를 준비한다.

    if not self.stateful or self.h is None:
        self.h = np.zeros((N, H), dtype='f')
        #h: RNN 계층의 은닉 상태. 
        #self.h=None: 처음 호출 시에는 원소가 모두 0인 영행렬로 초기화됨.
        #stateful=False: 항상 영행렬로 초기화

    for t in range(T):
        layer = RNN(*self.params)
        # *: 리스트의 원소들을 추출하여 메서드의 인수로 전달
        #self.params에 들어 있는 Wx, Wh, b를 추출하여 RNN 클래스의 __init__()메서드에 전달
        #RNN계층을 생성하여 인스턴스 변수 layers에 추가한다.
        self.h = layer.forward(xs[:, t, :], self.h)
        hs[:, t, :] = self.h
        self.layers.append(layer)
    
    return hs

#역전파 구현
def backward(self, dhs):
    Wx, Wh, b = self.params
    N, T, H = dhs.shape
    D, H = Wx.shape

    dxs = np.empty((N, T, D), dtype='f')
    dh = 0
    grads = [0, 0, 0]

    for t in reversed(range(T)):
        layer = self.layers[t]
        dx, dh = layer.backward(dhs[:, t, :] + dh) #합산된 기울기
        #RNN계층의 순전파에서는 출력이 2개로 분기되는데 역전파에서는 
        #각 기울기가 합산되어 전해진다.
        dxs[:, t, :] = dx

    for i, grad in enumerate(layer.grads):
        grads[i] +=grad

    for i, grad in enumerate(grads):
        self.grads[i][...] = grad
        self.dh = dh

    return dxs

### RNNLM 구현

In [11]:
import sys
sys.path.append('..')
import numpy as np
from common.time_layers import * 

#SimpleRnnlm클래스는 4개의 Time계층을 쌓은 신경망이다.
class SimpleRnnlm:
    def __init__(self, vocab_sizse, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        #가중치 초기화
        embed_W = (rn(V, D) / 100).astype('f')
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)), astype('f')
        affine_b = np.zeros(V).astype('f')
        #RNN 계층과 Affine계층에서 'Xabier초깃값'을 이용


        #계층 생성
        self.layers = [
        TimeEmbedding(embed_W),
        TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
        #Truncated BPTT로 학습한다고 가정하여 Time RNN계층의 stateful=True로 설정 -> TimeRNN계층은 이전 시간의 은닉 상태를 계승할 수 있다.
        TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]

        #모든 가중치와 기울기를 리스트에 모은다.
        self.params, self.grads = [],[]
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, ts):
        for layer in self.layers:
            xs = layer.forward(xs)
            loss = self.loss_layer.forward(xs, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        self.rnn_layer.reset_state()

### RNNLM 학습 코드

In [12]:
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
from common.optimizer import SGD
from dataset import ptb



# 하이퍼파라미터 설정
batch_size = 10
wordvec_size = 100
hidden_size = 100 # RNN의 은닉 상태 벡터의 원소 수
time_size = 5     # Truncated BPTT가 한 번에 펼치는 시간 크기
lr = 0.1
max_epoch = 100

# 학습 데이터 읽기(전체 중 1000개만)
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1]  # 입력
ts = corpus[1:]   # 출력(정답 레이블)
data_size = len(xs)
print('말뭉치 크기: %d, 어휘 수: %d' % (corpus_size, vocab_size))

# 학습 시 사용하는 변수
max_iters = data_size // (batch_size * time_size)
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

# 모델 생성
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)

# 미니배치의 각 샘플의 읽기 시작 위치를 계산
jump = (corpus_size - 1) // batch_size
offsets = [i * jump for i in range(batch_size)]
#각 미니배치가 데이터를 읽기 시작하는 위치를 계산해 offsets에 저장한다.
#offsets의 각 원소에 데이터를 읽는 시작 위치가 담기게 된다.

for epoch in range(max_epoch):
    for iter in range(max_iters):
        # 미니배치 취득
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
                #말뭉치를 읽는 위치가 말뭉치 크기를 넘어설 경우 말뭉치의 처음으로
                #돌아오기 위해서 말뭉치의 크기로 나눈 나머지를 인덱스로 사용한다.
            time_idx += 1
            #time_idx를 1씩 늘리면서 말뭉치에서 time_idx위치의 데이터를 얻는다.

        # 기울기를 구하여 매개변수 갱신
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1

    # 에폭마다 퍼플렉서티 평가
    ppl = np.exp(total_loss / loss_count)
    print('| 에폭 %d | 퍼플렉서티 %.2f'
          % (epoch+1, ppl))
    ppl_list.append(float(ppl))
    total_loss, loss_count = 0, 0

# 그래프 그리기
x = np.arange(len(ppl_list))
plt.plot(x, ppl_list, label='train')
plt.xlabel('epochs')
plt.ylabel('perplexity')
plt.show()

말뭉치 크기: 1000, 어휘 수: 418


NameError: name 'astype' is not defined

### RNNLM의 Trainer 클래스